In [22]:
import os
import sys
import platform
from datetime import datetime
from dotenv import load_dotenv

# Import the new LangGraph Brain
from langgraph.prebuilt import create_react_agent 
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate

# Import our Hybrid Switcher logic
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

load_dotenv()

# --- 1. SETUP THE BRAIN ---
def get_llm():
    llm_type = os.getenv("LLM_TYPE", "ollama")
    if llm_type == "groq":
        print(f"🚀 Brain: Groq (Fast)")
        return ChatGroq(model="llama-3.1-8b-instant", temperature=0)
    else:
        print(f"🏠 Brain: Local Ollama")
        return ChatOllama(model="llama3.1", temperature=0)

# --- 2. DEFINE THE TOOLS (THE HANDS) ---
@tool
def get_current_time():
    """Returns the current time in H:M:S format."""
    now = datetime.now()
    return now.strftime("%H:%M:%S")

@tool
def multiply_numbers(a: float, b: float):
    """Multiplies two numbers. Use this for ANY math calculation."""
    return a * b

@tool
def get_system_info():
    """Returns the type of OS"""
    system_info = platform.system()
    return system_info

# --- 3. BUILD THE AGENT ---

#The instruction manual for the Agent
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant.
    
    CORE RULES:
    1. You have tools (math, system info), but you should ONLY use them when specifically asked for calculations or system checks.
    2. For normal conversation (greetings, names, questions), just talk naturally.
    3. IMPORTANT: NEVER mention "JSON", "functions", or "tool calls" in your final response to the user. Just answer the question.
    """),
    ("placeholder", "{messages}") 
])

llm = get_llm()
tools = [get_current_time, multiply_numbers, get_system_info]


# This Function creates the "Loop" automatically
# It connects the LLM -> Tool -> Output -> LLM
memory = MemorySaver()
agent = create_react_agent(llm, tools, checkpointer=memory, prompt=prompt_template)

# --- 4. RUN THE AGENT ---
def run_agent(query, thread_id="default"):
    print(f"\nUser ({thread_id}): {query}")
    config = {"configurable": {"thread_id": thread_id}}
    
    # Notice: We trap errors just in case
    try:
        result = agent.invoke(
            {"messages": [HumanMessage(content=query)]},
            config=config
        )
        print(f"AI: {result['messages'][-1].content}")
    except Exception as e:
        print(f"❌ Error: {e}")

# Thread A: Captain Jack
run_agent("My name is Captain Jack.", thread_id="thread_1")
run_agent("What is my name?", thread_id="thread_1")

# Thread B: A completely different user
run_agent("My name is Will Turner.", thread_id="thread_2")

# Ask Thread B about Thread A (Should NOT know)
run_agent("What is the other guy's name?", thread_id="thread_2")

🏠 Brain: Local Ollama

User (thread_1): My name is Captain Jack.


/var/folders/s7/nglqvbh926vbsjnphgfq6k_h0000gn/T/ipykernel_58841/4271577039.py:69: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools, checkpointer=memory, prompt=prompt_template)


AI: I don't have any functions or tools to call for this question, so I'll just respond naturally.

Nice to meet you, Captain Jack! How can I assist you today?

User (thread_1): What is my name?
AI: Since you asked for a function call, I'll provide the response in the requested format. However, I must point out that there's no function provided to directly answer your question about your name.

But if I had to choose a function that could indirectly help with this question, I would use the "multiply_numbers" function is not relevant here, so I will use get_system_info and then somehow infer your name from it. 

{"name": "get_system_info", "parameters": {}}

User (thread_2): My name is Will Turner.
AI: I don't have any functions or tools to call for this question, so I'll just respond naturally.

Nice to meet you, Will! How can I assist you today?

User (thread_2): What is the other guy's name?
AI: I don't have any functions or tools to call for this question, so I'll just respond natur